In [1]:
# Country (https://countrycode.org/)
country_url = "https://countrycode.org/"
# State (US: http://www.50states.com/abbreviations.htm, Canada: http://www.comeexplorecanada.com/abbreviations.php)
state_us_url = "http://www.50states.com/abbreviations.htm"
state_ca_url = "http://www.ncbi.nlm.nih.gov/books/NBK7254/"

In [2]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import pymongo

In [3]:
def get_page_soup(website_url):
    page = requests.get(website_url)
    if not page.ok: # if the page was returned ok
        return None

    soup = BeautifulSoup(page.text, "html.parser")
    return soup

In [12]:
def insert_to_db(data_to_insert, collection_name, index_name):
    connection = MongoClient('localhost', 27017)
    db = connection.hacktown_utils

    response = db[collection_name].insert(data_to_insert)
    
    if response:
        index_res = db[collection_name].ensure_index(index_name, pymongo.ASCENDING, unique=True)
        
    connection.close()
    
    return response, index_res

In [5]:
# get country list and abbreviations (2/3 letter) page

soup = get_page_soup(country_url)
if soup:
    print "OK"

OK


/proj/virtualenvs/hacktown/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [6]:
all_countries =  soup.tbody.select("tr")
countries_list = []
for country in all_countries:
    all_abbvs = country.select("td")[2].string.strip().split('/')
    country_dict = {"name": country.a.string.strip(),
                   "abbreviation_2_letter":all_abbvs[0].strip(),
                   "abbreviation_3_letter":all_abbvs[1].strip(),
                   "type":"Country"}
    countries_list.append(country_dict)
    
len(countries_list)

240

In [ ]:
insert_to_db(countries_list, "all_country_abbreviations", "name")

In [14]:
# get US states list and abbreviations

soup = get_page_soup(state_us_url)
if soup:
    print "OK"

OK


In [15]:
all_states_us =  soup.table.select("tr")
state_us_list = []
for state in all_states_us:
    if state.td:
        state_dict = {"name":state.td.string.strip(), 
                      "abbreviation_2_letter":state.select("td")[1].string.strip(),
                     "country": "United States",
                     "type":"State"}
        
        state_us_list.append(state_dict)
len(state_us_list)

65

In [ ]:
insert_to_db(state_us_list, "all_subcountry_abbreviations", "name")

In [17]:
# get CA provinces and abbreviations

soup = get_page_soup(state_ca_url)
if soup:
    print "OK"

OK


In [18]:
all_provinces =  soup.tbody.select("tr")
all_province_list = []
for province in all_provinces:
    province_dict = {"name":province.select("td")[0].string.strip(),
                    "abbreviation_2_letter" : province.select("td")[1].string.strip(),
                    "country":"Canada",
                    "type":"Province"}
    
    all_province_list.append(province_dict)
    
print (all_province_list)

[{'country': 'Canada', 'type': 'Province', 'name': u'Alberta', 'abbreviation_2_letter': u'AB'}, {'country': 'Canada', 'type': 'Province', 'name': u'British Columbia', 'abbreviation_2_letter': u'BC'}, {'country': 'Canada', 'type': 'Province', 'name': u'Manitoba', 'abbreviation_2_letter': u'MB'}, {'country': 'Canada', 'type': 'Province', 'name': u'New Brunswick', 'abbreviation_2_letter': u'NB'}, {'country': 'Canada', 'type': 'Province', 'name': u'Newfoundland and Labrador', 'abbreviation_2_letter': u'NL'}, {'country': 'Canada', 'type': 'Province', 'name': u'Northwest Territories', 'abbreviation_2_letter': u'NT'}, {'country': 'Canada', 'type': 'Province', 'name': u'Nova Scotia', 'abbreviation_2_letter': u'NS'}, {'country': 'Canada', 'type': 'Province', 'name': u'Nunavut', 'abbreviation_2_letter': u'NU'}, {'country': 'Canada', 'type': 'Province', 'name': u'Ontario', 'abbreviation_2_letter': u'ON'}, {'country': 'Canada', 'type': 'Province', 'name': u'Prince Edward Island', 'abbreviation_2_l

In [ ]:
insert_to_db(all_province_list, "all_subcountry_abbreviations", "name")